In [2]:
# 显示cell运行时长
!pip install -i https://pypi.doubanio.com/simple/  --trusted-host pypi.doubanio.com pytorch_toolbelt timm cnn_finetune opencv-python

Looking in indexes: https://pypi.doubanio.com/simple/
     |████████████████████████████████| 92 kB 772 kB/s eta 0:00:011
     |████████████████████████████████| 225 kB 1.8 MB/s eta 0:00:01
     |████████████████████████████████| 49.5 MB 1.4 kB/s eta 0:00:011:01��██                          | 9.3 MB 90.4 MB/s eta 0:00:01      | 17.6 MB 90.4 MB/s eta 0:00:01��████████▌                   | 19.3 MB 90.4 MB/s eta 0:00:01��█████████                   | 20.0 MB 90.4 MB/s eta 0:00:01███▍                 | 22.3 MB 90.4 MB/s eta 0:00:01██████████▏             | 28.1 MB 757 kB/s eta 0:00:29kB/s eta 0:00:26��██▍           | 31.5 MB 757 kB/s eta 0:00:24��███▎    | 42.1 MB 798 kB/s eta 0:00:10ta 0:00:03
     |████████████████████████████████| 58 kB 2.5 MB/s eta 0:00:011
     |████████████████████████████████| 829 kB 86.8 MB/s eta 0:00:01
     |████████████████████████████████| 676 kB 88.6 MB/s eta 0:00:01
     |████████████████████████████████| 200 kB 1.6 MB/s eta 0:00:01
  Created wheel for pytorc

In [8]:
#
'''
import json
import os
import random
#制作训练集和验证集
with open('/home/kesci/data/competition/train_set/Part1.json','r') as f:
    part1_data=json.load(f)
with open('/home/kesci/data/competition/train_set/Part2.json','r') as f:
    part2_data=json.load(f)
with open('/home/kesci/data/competition/train_set/Part3.json','r') as f:
    part3_data=json.load(f)
with open('/home/kesci/data/competition/train_set/Part4.json','r') as f:
    part4_data=json.load(f)
#
f_train_val=[]
f_val=open('/home/kesci/work/dataset/val_pseudo.txt','w')
f_train=open('/home/kesci/work/dataset/train_pseudo.txt','w')
class_label={'sunny':'0','cloudy':'1','others':'2'}
cnt_classes={'sunny':0,'cloudy':0,'others':0}
for img_name in part1_data.keys():
    cnt_classes[part1_data[img_name]]+=1
    f_train_val.append('train_set/Part1/'+img_name+','+class_label[part1_data[img_name]]+'\n')
    #f_train_val.write('Part1/'+img_name+','+class_label[part1_data[img_name]]+'\n')
for img_name in part2_data.keys():
    cnt_classes[part2_data[img_name]]+=1
    f_train_val.append('train_set/Part2/'+img_name+','+class_label[part2_data[img_name]]+'\n')
    #f_train_val.write('Part2/'+img_name+','+class_label[part2_data[img_name]]+'\n')
for img_name in part3_data.keys():
    cnt_classes[part3_data[img_name]]+=1
    f_train_val.append('train_set/Part3/'+img_name+','+class_label[part3_data[img_name]]+'\n')
    #f_train_val.write('Part3/'+img_name+','+class_label[part3_data[img_name]]+'\n')
for img_name in part4_data.keys():
    cnt_classes[part4_data[img_name]]+=1
    f_train_val.append('train_set/Part4/'+img_name+','+class_label[part4_data[img_name]]+'\n')
    #f_train_val.write('Part4/'+img_name+','+class_label[part4_data[img_name]]+'\n')
#split train/val
train_size=0
test_size=0
for i in range(len(f_train_val)):
    if i%17==0:
        test_size+=1
        f_val.write(f_train_val[i])
    else:
        f_train.write(f_train_val[i])
        train_size+=1
f_train.close()
f_val.close()
print("train_size:{};test_size:{}".format(train_size,test_size))
cnt_classes
'''

train_size:169020;test_size:10564


{'sunny': 61207, 'cloudy': 43614, 'others': 74763}

In [2]:
169020+52793

221813

In [9]:
import numpy as np
cnt_cls=np.array([61270,43614,74763])
cnt_cls/np.sum(cnt_cls)

array([0.34105774, 0.24277611, 0.41616615])

In [4]:
import os
import glob
from PIL import Image
import torch
from torch.utils.data import Dataset,DataLoader
import numpy as np
from torchvision import transforms as T
import torchvision
import cv2
import pandas as pd
import torch.nn as nn
from torch.utils.data import DataLoader,Dataset
import torch.optim as optim
import time
from sklearn.metrics import accuracy_score,f1_score,precision_recall_fscore_support
from cnn_finetune import make_model
from torch.optim.lr_scheduler import CosineAnnealingLR, StepLR
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
class Config(object):
    backbone = 'wsl'#'inception_v4'
    num_classes = 52 #
    use_arcLoss = False
    metric = 'arc_margin'
    easy_margin = False
    loss = 'CrossEntropyLoss'#focal_loss/CrossEntropyLoss
    feature_dimension=128
    #
    input_size = 300
    train_batch_size = 24  # batch size
    val_batch_size = 4
    test_batch_size = 1
    optimizer = 'sgd'
    lr_scheduler='cosine'
    lr = 1e-4  # adam 0.00001
    MOMENTUM = 0.9
    device = "cuda"  # cuda  or cpu
    gpu_id = 0
    num_workers = 4  # how many workers for loading data
    max_epoch = 30
    lr_decay_epoch = 10
    lr_decay = 0.95  # when val_loss increase, lr = lr*lr_decay
    weight_decay = 5e-4
    val_interval = 1
    print_interval = 50
    save_interval = 1
    min_save_epoch=0
    #
    log_dir = 'log/'
    train_val_data='/home/kesci/data1/competition/train_set'
    train_list='/home/kesci/work/dataset/train.txt'
    val_list='/home/kesci/work/dataset/val.txt'
    #
    test_dir='/home/kesci/data1/competition/test_set/'
    submit_csv_dir='/home/kesci/work/submit/'
    #
    checkpoints_dir = '/home/kesci/work/checkpoints/'
    test_model_dir= '/home/kesci/work/checkpoints/wsl/wsl_1.pth'
#
class wheatherDataset(Dataset):
    def __init__(self, root, data_list_file, phase='train', input_size=640):
        self.phase = phase

        with open(os.path.join(data_list_file), 'r') as fd:
            imgs = fd.readlines()

        imgs = [os.path.join(root, img.strip('\n')) for img in imgs]
        self.imgs = np.random.permutation(imgs)

        normalize = T.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))


        if self.phase == 'train':
            self.transforms = T.Compose([
                T.Resize((input_size,input_size)),
                #T.RandomHorizontalFlip(p=0.5),
                T.ToTensor(),
                normalize
            ])
        else:
            self.transforms = T.Compose([
                T.Resize((input_size,input_size)),
                T.ToTensor(),
                normalize
            ])

    def __getitem__(self, index):
        sample = self.imgs[index]
        splits = sample.split(',')
        img_path = splits[0]
        data = Image.open(img_path)
        data = data.convert('RGB')
        data = self.transforms(data)
        label = np.int32(splits[1])
        return data.float(), label

    def __len__(self):
        return len(self.imgs)

class wheatherDatasetInfer(Dataset):

    def __init__(self,data_dir, input_size=300,transform=None):
        self.image_paths = sorted(glob.glob(data_dir+'/*'))
        self.transforms = T.Compose([
                T.Resize((input_size,input_size)),
                T.RandomHorizontalFlip(p=0.5),
                T.ToTensor(),
                T.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
            ])

    def __getitem__(self, index):
        sample_path = self.image_paths[index]
        data = Image.open(sample_path)
        data = data.convert('RGB')
        data = self.transforms(data)
        return data.float(),sample_path

    def __len__(self):
        return len(self.image_paths)

if __name__ == '__main__':
    opt=Config()
    dataset = wheatherDataset(root=opt.train_val_data,
                      data_list_file=opt.val_list,
                      phase='train',
                      input_size=opt.input_size)

    trainloader = DataLoader(dataset, batch_size=2)
    for i, (data, label) in enumerate(trainloader):
        # imgs, labels = data
        # print imgs.numpy().shape
        # print data.cpu().numpy()
        # if i == 0:
        print(data.shape)
        img = torchvision.utils.make_grid(data).numpy()
        # print img.shape
        # print label.shape
        # chw -> hwc
        img = np.transpose(img, (1, 2, 0))
        #cv2.imshow('img', img)
        img *= np.array([0.5, 0.5, 0.5])*255
        img += np.array([0.5, 0.5, 0.5])*255
        #img += np.array([1, 1, 1])
        #img *= 127.5
        img = img.astype(np.uint8)
        img = img[:, :, [2, 1, 0]]

        print(img.shape)
        #cv2.imshow('img', img)
        #cv2.waitKey(10000)
        break
        # dst.decode_segmap(labels.numpy()[0], plot=True)
#

import torch
import torch.nn as nn


class LabelSmoothSoftmaxCE(nn.Module):
    def __init__(self,
                 lb_pos=0.9,
                 lb_neg=0.005,
                 reduction='mean',
                 lb_ignore=255,
                 ):
        super(LabelSmoothSoftmaxCE, self).__init__()
        self.lb_pos = lb_pos
        self.lb_neg = lb_neg
        self.reduction = reduction
        self.lb_ignore = lb_ignore
        self.log_softmax = nn.LogSoftmax(1)

    def forward(self, logits, label):
        logs = self.log_softmax(logits)
        ignore = label.data.cpu() == self.lb_ignore
        n_valid = (ignore == 0).sum()
        label = label.clone()
        label[ignore] = 0
        lb_one_hot = logits.data.clone().zero_().scatter_(1, label.unsqueeze(1), 1)
        label = self.lb_pos * lb_one_hot + self.lb_neg * (1-lb_one_hot)
        ignore = ignore.nonzero()
        _, M = ignore.size()
        a, *b = ignore.chunk(M, dim=1)
        label[[a, torch.arange(label.size(1)), *b]] = 0

        if self.reduction == 'mean':
            loss = -torch.sum(torch.sum(logs*label, dim=1)) / n_valid
        elif self.reduction == 'none':
            loss = -torch.sum(logs*label, dim=1)
        return loss
import logging

def get_logger(filename, verbosity=1, name=None):
    level_dict = {0: logging.DEBUG, 1: logging.INFO, 2: logging.WARNING}
    formatter = logging.Formatter(
        "[%(asctime)s][%(filename)s][%(levelname)s] %(message)s"
    )
    logger = logging.getLogger(name)
    logger.setLevel(level_dict[verbosity])

    fh = logging.FileHandler(filename, "w")
    fh.setFormatter(formatter)
    logger.addHandler(fh)

    sh = logging.StreamHandler()
    sh.setFormatter(formatter)
    logger.addHandler(sh)
    return logger
#

torch.Size([2, 3, 300, 300])
(304, 606, 3)


In [3]:
#因为xception权重自动下载太慢，所以先本地下载好，再按照挂载数据那样传上去，再复制到指定目录下,其它模型相同操作
#!cp /home/kesci/input/pre_train5544/xception-43020ad28.pth /home/kesci/.cache/torch/hub/checkpoints/

In [4]:
#!cp /home/kesci/input/swsr5567/semi_weakly_supervised_resnext101_32x8-b4712904.pth /home/kesci/.cache/torch/hub/checkpoints

## 实验记录
- 300输入尺度，5e-4学习率，水平翻转:一个epoch:0.829,线下0.808
- 0.829模型用作预训练，2e-4学习率，再训一个epoch，线上0.842，线下0.818
- 加上标签平滑(双loss)，将8424的作为预训练再训练一个epoch,线下0.819(812~819波动)
    - 虽然都为线下818,但是（三类）侧重点却不一样，分别交的结果都有842，所以训练三个模型进行一个投票操作.线上0.845

In [5]:
#from pytorch_toolbelt import losses as L
#!pip install pytorch_toolbelt

In [5]:
#
import timm
from pytorch_toolbelt import losses as L
def train_model(model,criterion, optimizer, lr_scheduler,arc_model=None):

    train_dataset = wheatherDataset(opt.train_val_data, opt.train_list, phase='train', input_size=opt.input_size)
    trainloader = DataLoader(train_dataset,
                             batch_size=opt.train_batch_size,
                             shuffle=True,
                             num_workers=opt.num_workers)

    total_iters=len(trainloader)
    logger.info('total_iters:{}'.format(total_iters))
    model_name=opt.backbone
    train_loss = []
    since = time.time()
    best_model_wts = model.state_dict()
    best_score = 0.0
    model.train(True)
    logger.info('start training...')
    #
    iters = len(trainloader)
    for epoch in range(1,opt.max_epoch+1):
        begin_time=time.time()
        logger.info('learning rate:{}'.format(optimizer.param_groups[-1]['lr']))
        logger.info('Epoch {}/{}'.format(epoch, opt.max_epoch))
        logger.info('-' * 10)
        optimizer = lr_scheduler(optimizer, epoch)
        running_loss = 0.0
        running_corrects_linear = 0
        running_corrects_arc=0
        count=0
        best_epoch=0
        for i, data in enumerate(trainloader):
            count+=1
            inputs, labels = data
            labels = labels.type(torch.LongTensor)
            inputs, labels = inputs.cuda(), labels.cuda()
            #
            out_linear= model(inputs)
            _, linear_preds = torch.max(out_linear.data, 1)
            loss = criterion(out_linear, labels)
            #
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if i % opt.print_interval == 0 or out_linear.size()[0] < opt.train_batch_size:
                spend_time = time.time() - begin_time
                logger.info(
                    ' Epoch:{}({}/{}) loss:{:.3f} lr:{:.7f} epoch_Time:{}min:'.format(
                        epoch, count, total_iters,
                        loss.item(), optimizer.param_groups[-1]['lr'],
                        spend_time / count * total_iters // 60 - spend_time // 60))
                train_loss.append(loss.item())
            running_corrects_linear += torch.sum(linear_preds == labels.data)
            #
        weight_score = val_model(model, criterion)
        epoch_acc_linear = running_corrects_linear.double() / total_iters / opt.train_batch_size
        logger.info('Epoch:[{}/{}] train_acc={:.3f} '.format(epoch, opt.max_epoch,
                                                                    epoch_acc_linear))
        save_dir = os.path.join(opt.checkpoints_dir, model_name)
        if not os.path.exists(save_dir):
            os.makedirs(save_dir)
        model_out_path = save_dir + "/" + '{}_'.format(model_name) + str(epoch) + '.pth'
        best_model_out_path = save_dir + "/" + '{}_'.format(model_name) + 'best' + '.pth'
        #save the best model
        if weight_score > best_score:
            best_score = weight_score
            best_epoch=epoch
            torch.save(model.state_dict(), best_model_out_path)
        #save based on epoch interval
        if epoch % opt.save_interval == 0 and epoch>opt.min_save_epoch:
            torch.save(model.state_dict(), model_out_path)
    #
    #optimizer.swap_swa_sgd
    #
    logger.info('Best WeightF1: {:.3f} Best epoch:{}'.format(best_score,best_epoch))
    time_elapsed = time.time() - since
    logger.info('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))


def val_model(model, criterion):
    val_dataset = wheatherDataset(opt.train_val_data, opt.val_list, phase='val', input_size=opt.input_size)
    val_loader = DataLoader(val_dataset,
                             batch_size=opt.val_batch_size,
                             shuffle=False,
                             num_workers=opt.num_workers)
    dset_sizes=len(val_dataset)
    model.eval()
    running_loss = 0.0
    running_corrects = 0
    cont = 0
    outPre = []
    outLabel = []
    pres_list=[]
    labels_list=[]
    for data in val_loader:
        inputs, labels = data
        labels = labels.type(torch.LongTensor)
        inputs, labels = inputs.cuda(), labels.cuda()
        outputs = model(inputs)
        _, preds = torch.max(outputs.data, 1)
        loss = criterion(outputs, labels)
        if cont == 0:
            outPre = outputs.data.cpu()
            outLabel = labels.data.cpu()
        else:
            outPre = torch.cat((outPre, outputs.data.cpu()), 0)
            outLabel = torch.cat((outLabel, labels.data.cpu()), 0)
        pres_list+=preds.cpu().numpy().tolist()
        labels_list+=labels.data.cpu().numpy().tolist()
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)
        cont += 1
    _,_, f_class, _= precision_recall_fscore_support(y_true=labels_list, y_pred=pres_list,labels=[0, 1, 2],
     average=None)                                                                   
    fper_class = {'sunny': f_class[0], 'cloudy': f_class[1], 'others': f_class[2]}
    logger.info('各类单独F1:{}  各类F1取平均:{}'.format(fper_class, f_class.mean()))
    logger.info('val_size: {}  valLoss: {:.4f} valAcc: {:.4f}'.format(dset_sizes, running_loss / dset_sizes, running_corrects.double() / dset_sizes))
    val_acc = accuracy_score(labels_list, pres_list)    
    return val_acc


def exp_lr_scheduler(optimizer, epoch):
    #lr = opt.lr * (0.8**(epoch / opt.lr_decay_epoch))
    lr = opt.lr * 0.5**(epoch-1)
    logger.info('Learning Rate is {:.5f}'.format(lr))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr
    return optimizer

if __name__ == "__main__":
    #
    opt = Config()
    torch.cuda.empty_cache()
    device = torch.device(opt.device)
    #if opt.loss == 'focal_loss':
        #criterion = FocalLoss(gamma=2)
    #else:
    #weight_class=torch.FloatTensor([0.34, 0.24, 0.42])
    #criterion = torch.nn.CrossEntropyLoss(weight=weight_class).cuda()
    criterion = torch.nn.CrossEntropyLoss().cuda()
    use_smooth_label=False
    if use_smooth_label:
        criterion = L.JointLoss(first=torch.nn.CrossEntropyLoss(weight=weight_class), second=LabelSmoothSoftmaxCE(),
                              first_weight=0.5, second_weight=0.5).cuda()
    #
    model_name =opt.backbone
    logger = get_logger(os.path.join(opt.checkpoints_dir , model_name+'.log'))
    logger.info('Using: {}'.format(model_name))
    logger.info('InputSize: {}'.format(opt.input_size))
    logger.info('optimizer: {}'.format(opt.optimizer))
    logger.info('lr_init: {}'.format(opt.lr))
    logger.info('criterion: {}'.format(opt.loss))
    logger.info('Using the GPU: {}'.format(str(opt.gpu_id)))

    #model  = make_model('{}'.format(model_name), num_classes=3,
    #                    pretrained=True)
    model = timm.create_model('swsl_resnext101_32x8d', pretrained=True)
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, 3)
    model.to(device)
    model = nn.DataParallel(model)
    #pretrained
    net_weight = torch.load('/home/kesci/work/checkpoints/wsl_83/wsl_1_83.pth')
    #net_weight = torch.load('/home/kesci/work/checkpoints/wsl_84/wsl_1_84.pth')
    model.load_state_dict(net_weight)
    if opt.optimizer == 'sgd':
        optimizer = optim.SGD((model.parameters()), lr=opt.lr, momentum=opt.MOMENTUM, weight_decay=0.0004)
    else:
        optimizer = optim.Adam(model.parameters(), lr=opt.lr)

    train_model(model, criterion, optimizer,
              lr_scheduler=exp_lr_scheduler)

    torch.cuda.empty_cache()


[2020-10-22 13:42:42,949][<ipython-input-6-4a7b177564e6>][INFO] Using: wsl
[2020-10-22 13:42:42,950][<ipython-input-6-4a7b177564e6>][INFO] InputSize: 300
[2020-10-22 13:42:42,951][<ipython-input-6-4a7b177564e6>][INFO] optimizer: sgd
[2020-10-22 13:42:42,951][<ipython-input-6-4a7b177564e6>][INFO] lr_init: 0.0001
[2020-10-22 13:42:42,952][<ipython-input-6-4a7b177564e6>][INFO] criterion: CrossEntropyLoss
[2020-10-22 13:42:42,953][<ipython-input-6-4a7b177564e6>][INFO] Using the GPU: 0
[2020-10-22 13:42:51,732][<ipython-input-6-4a7b177564e6>][INFO] total_iters:7043
[2020-10-22 13:42:51,737][<ipython-input-6-4a7b177564e6>][INFO] start training...
[2020-10-22 13:42:51,738][<ipython-input-6-4a7b177564e6>][INFO] learning rate:0.0001
[2020-10-22 13:42:51,738][<ipython-input-6-4a7b177564e6>][INFO] Epoch 1/30
[2020-10-22 13:42:51,738][<ipython-input-6-4a7b177564e6>][INFO] ----------
[2020-10-22 13:42:51,739][<ipython-input-6-4a7b177564e6>][INFO] Learning Rate is 0.00010
[2020-10-22 13:42:54,445][<

In [ ]:
1+1

In [5]:
#infer
import tqdm
import timm
class wheatherDatasetInfer(Dataset):

    def __init__(self,data_dir, input_size=300,transform=None):
        self.image_paths = sorted(glob.glob(data_dir+'/*'))
        self.transforms = T.Compose([
                T.Resize((input_size,input_size)),
                T.ToTensor(),
                T.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
            ])

    def __getitem__(self, index):
        sample_path = self.image_paths[index]
        data = Image.open(sample_path)
        data = data.convert('RGB')
        data = self.transforms(data)
        return data.float(),sample_path

    def __len__(self):
        return len(self.image_paths)
@torch.no_grad()
def infer_wheather(model):
    pres_dic = {'id':[],'weather':[]}
    cnt = 0
    for data in dataset_loaders:
        if cnt % 100 == 0:
            print(cnt)
        cnt += 1
        inputs, image_paths = data
        inputs = inputs.cuda()
        outputs = model(inputs)
        #out_prob = F.softmax(outputs)[0]
        _, preds = torch.max(outputs.data, 1)
        cls = preds.cpu().numpy().tolist()
        image_names=[per.split('/')[-1] for per in image_paths]
        #print(image_names)
        pres_dic['id']+=image_names
        pres_dic['weather']+=cls
    #
    df_sub=pd.DataFrame(pres_dic)
    df_sub.to_csv(opt.submit_csv_dir+'10_23_821.csv',index=False)



if __name__ == '__main__':
    opt = Config()
    # some parameters
    use_gpu = torch.cuda.is_available()
    os.environ["CUDA_VISIBLE_DEVICES"] = "0"

    image_datasets = wheatherDatasetInfer(opt.test_dir,opt.input_size)
    #
    dataset_loaders = torch.utils.data.DataLoader(image_datasets,
                                                  batch_size=32,
                                                  shuffle=False, num_workers=4)
    data_set_sizes = len(image_datasets)
    print('测试数据量:{},batch数:{}'.format(data_set_sizes,data_set_sizes//32))
    #
    #model = make_model('{}'.format(opt.backbone), num_classes=3,
                         #pretrained=False)
    model = timm.create_model('swsl_resnext101_32x8d', pretrained=False)
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, 3)
    #
    device = torch.device(opt.device)
    model.to(device)
    model = nn.DataParallel(model)
    net_weight = torch.load('/home/kesci/work/checkpoints/wsl/wsl_1.pth')
    model.load_state_dict(net_weight)
    model.eval()
    print('download model finished....')
    #
    infer_wheather(model)


测试数据量:72778,batch数:2274
download model finished....
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200


In [8]:
#infer 三模融合
import timm
from torch.nn import functional as F
class wheatherDatasetInfer(Dataset):

    def __init__(self,data_dir, input_size=300,transform=None):
        self.image_paths = sorted(glob.glob(data_dir+'/*'))
        self.transforms = T.Compose([
                T.Resize((input_size,input_size)),
                T.ToTensor(),
                T.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
            ])

    def __getitem__(self, index):
        sample_path = self.image_paths[index]
        data = Image.open(sample_path)
        data = data.convert('RGB')
        data = self.transforms(data)
        return data.float(),sample_path

    def __len__(self):
        return len(self.image_paths)
@torch.no_grad()
def infer_wheather(model_1,model_2,model_3):
    pres_dic = {'id':[],'weather':[]}
    cnt = 0
    all_pathes=[]
    all_outs=[]
    for data in dataset_loaders:
        if cnt % 100 == 0:
            print(cnt)
        cnt += 1
        inputs, image_paths = data
        inputs = inputs.cuda()
        outputs1 = model_1(inputs)
        outputs2 = model_2(inputs)
        outputs3 = model_3(inputs)
        outputs=(outputs1+outputs2+outputs3)/3#算数平均
        all_outs.append(F.softmax(outputs))
        #outputs_geom=torch.pow(outputs1*outputs2*outputs3,1/3)#几何平均
        _, preds = torch.max(outputs.data, 1)
        cls = preds.cpu().numpy().tolist()
        image_names=[per.split('/')[-1] for per in image_paths]
        #print(image_names)
        pres_dic['id']+=image_names
        pres_dic['weather']+=cls
        all_pathes.append(image_names)
    #
    df_sub=pd.DataFrame(pres_dic)
    #df_sub.to_csv(opt.submit_csv_dir+'10_20v2Mean3.csv',index=False)
    return all_pathes,all_outs


if __name__ == '__main__':
    opt = Config()
    # some parameters
    use_gpu = torch.cuda.is_available()
    os.environ["CUDA_VISIBLE_DEVICES"] = "0"

    image_datasets = wheatherDatasetInfer(opt.test_dir,opt.input_size)
    #
    dataset_loaders = torch.utils.data.DataLoader(image_datasets,
                                                  batch_size=32,
                                                  shuffle=False, num_workers=4)
    data_set_sizes = len(image_datasets)
    print('测试数据量:{},batch数:{}'.format(data_set_sizes,data_set_sizes//32))
    #
    #model = make_model('{}'.format(opt.backbone), num_classes=3,
                         #pretrained=False)
    device = torch.device(opt.device)
    model_1 = timm.create_model('swsl_resnext101_32x8d', pretrained=False)
    num_ftrs = model_1.fc.in_features
    model_1.fc = nn.Linear(num_ftrs, 3)
    #
    model_1.to(device)
    model_1 = nn.DataParallel(model_1)
    net_weight_1 = torch.load('/home/kesci/work/checkpoints/wsl_84/wsl_1_84.pth')
    model_1.load_state_dict(net_weight_1)
    model_1.eval()
    print('download model1 finished....')
    #
    model_2 = timm.create_model('swsl_resnext101_32x8d', pretrained=False)
    num_ftrs = model_2.fc.in_features
    model_2.fc = nn.Linear(num_ftrs, 3)
    model_2.to(device)
    model_2 = nn.DataParallel(model_2)
    net_weight_2 = torch.load('/home/kesci/work/checkpoints/wsl_84244/wsl_1.pth')
    model_2.load_state_dict(net_weight_2)
    model_2.eval()
    print('download model2 finished....')
    #
    #
    model_3 = timm.create_model('swsl_resnext101_32x8d', pretrained=False)
    num_ftrs = model_3.fc.in_features
    model_3.fc = nn.Linear(num_ftrs, 3)
    model_3.to(device)
    model_3 = nn.DataParallel(model_3)
    net_weight_3 = torch.load('/home/kesci/work/checkpoints/wsl_84_v1/wsl_1.pth')
    model_3.load_state_dict(net_weight_3)
    model_3.eval()
    print('download model3 finished....')
    all_pathes,all_outs=infer_wheather(model_1,model_2,model_3)


测试数据量:72778,batch数:2274
download model1 finished....
download model2 finished....
download model3 finished....
0


<ipython-input-8-8aa2ef523ef4>:39: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  all_outs.append(F.softmax(outputs))


100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200


52797

In [20]:
'''
reserve_list=[]#按照概率进行伪标签采样
cnt_pseudo_df={'image_name':[],'pro':[]}
for i in range(len(all_pathes)):
    out_pre=all_outs[i]
    max_pro, preds = torch.max(out_pre, 1)
    max_pro_list = max_pro.cpu().numpy().tolist()
    for jj in range(len(max_pro_list)):
        #print(max_pro_list[jj])
        cnt_pseudo_df['pro'].append(max_pro_list[jj])
        cnt_pseudo_df['image_name'].append(all_pathes[i][jj])
        if max_pro_list[jj]>0.8:
            reserve_list.append(all_pathes[i][jj])
len(reserve_list)
import pandas as pd
cnt_pseudo_df=pd.DataFrame(cnt_pseudo_df)
cnt_pseudo_df.to_csv('/home/kesci/work/submit/cnt_pseudo_df.csv',index=False)
'''

In [11]:
#3模融合(投票)

result_1=pd.read_csv('/home/kesci/work/submit/10_20v1_844.csv')
result_2=pd.read_csv('/home/kesci/work/submit/10_20v2_845.csv')
result_3=pd.read_csv('/home/kesci/work/submit/10_18v1.csv')
result_4=pd.read_csv('/home/kesci/work/submit/10_20v2Mean3.csv')
sub_1=result_1['weather'].values
sub_2=result_2['weather'].values
sub_3=result_3['weather'].values
sub_4=result_4['weather'].values
sub_5=result_5['weather'].values
result_list=[]
for i in range(len(sub_1)):
    tmp = {0: 0, 1: 0, 2: 0}
    votes=[sub_1[i],sub_2[i],sub_3[i],sub_4[i],sub_4[i],sub_4[i]]
    for k in votes:
        tmp[k] += 1
    #
    most = sorted(tmp.items(), key=lambda item: item[1])[-1][0]
    result_list.append(most)
#
result=result_1.copy()
result['weather']=result_list
result.to_csv('/home/kesci/work/submit/10_22voteV2.csv',index=False)



In [16]:
reserve_list=[]
df_tmp=pd.read_csv('/home/kesci/work/submit/cnt_pseudo_df.csv')
for index,img_name,pro in df_tmp.itertuples():
    if pro>0.8:
        reserve_list.append(img_name)
#
#pseudo-label 利用线上8456的模型做一个伪标签
best_true=pd.read_csv('/home/kesci/work/submit/10_20voteV2_8456.csv')
train_txt='/home/kesci/work/dataset/train_pseudo.txt'
w_contine=open(train_txt,'a')#追加测试集的标签

print('pseudo len:',len(reserve_list))
cnt=0
for index,img_name,value in best_true.itertuples():
    if img_name in reserve_list:
        w_contine.write('test_set/'+img_name+','+str(value)+'\n')
        cnt+=1
    #print(img_name,value)
w_contine.close()
print('伪标签制作完成',cnt)



pseudo len: 52797
伪标签制作完成 52797


In [6]:
#infer
#尝试TTA
'''
import tqdm
import timm
class wheatherDatasetInfer(Dataset):

    def __init__(self,data_dir, input_size=300,modes=0):
        self.image_paths = sorted(glob.glob(data_dir+'/*'))

        
        self.transforms= T.Compose([
                    T.Resize((input_size,input_size)),
                    T.RandomHorizontalFlip(p=1),
                    T.ToTensor(),#T.RandomHorizontalFlip(p=1),
                    T.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
                ])

    def __getitem__(self, index):
        sample_path = self.image_paths[index]
        data = Image.open(sample_path)
        data = data.convert('RGB')
        data = self.transforms(data)
        return data.float(),sample_path

    def __len__(self):
        return len(self.image_paths)
#infer
import tqdm
import timm
@torch.no_grad()
def infer_wheather(model):
    pres_dic = {'id':[],'weather':[]}
    cnt = 0
    all_output=[]
    all_pathes=[]
    for data in dataset_loaders:
        if cnt % 100 == 0:
            print(cnt)
        cnt += 1
        inputs, image_paths = data
        inputs = inputs.cuda()
        outputs = model(inputs)
        all_output.append(outputs)
        image_names=[per.split('/')[-1] for per in image_paths]
        all_pathes.append(image_names)
    #
    return all_pathes,all_output



if __name__ == '__main__':
    opt = Config()
    # some parameters
    use_gpu = torch.cuda.is_available()
    os.environ["CUDA_VISIBLE_DEVICES"] = "0"

    image_datasets = wheatherDatasetInfer(opt.test_dir,opt.input_size)
    #
    dataset_loaders = torch.utils.data.DataLoader(image_datasets,
                                                  batch_size=48,
                                                  shuffle=False, num_workers=8)
    data_set_sizes = len(image_datasets)
    print('测试数据量:{},batch数:{}'.format(data_set_sizes,data_set_sizes//48))
    #
    #model = make_model('{}'.format(opt.backbone), num_classes=3,
                         #pretrained=False)
    model = timm.create_model('swsl_resnext101_32x8d', pretrained=False)
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, 3)
    #
    device = torch.device(opt.device)
    model.to(device)
    model = nn.DataParallel(model)
    net_weight = torch.load('/home/kesci/work/checkpoints/wsl/wsl_1.pth')
    model.load_state_dict(net_weight)
    model.eval()
    print('download model finished....')
    #
    all_pathes_1_1,all_output_2_1=infer_wheather(model)
'''

'\nimport tqdm\nimport timm\nclass wheatherDatasetInfer(Dataset):\n\n    def __init__(self,data_dir, input_size=300,modes=0):\n        self.image_paths = sorted(glob.glob(data_dir+\'/*\'))\n\n        \n        self.transforms= T.Compose([\n                    T.Resize((input_size,input_size)),\n                    T.RandomHorizontalFlip(p=1),\n                    T.ToTensor(),#T.RandomHorizontalFlip(p=1),\n                    T.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))\n                ])\n\n    def __getitem__(self, index):\n        sample_path = self.image_paths[index]\n        data = Image.open(sample_path)\n        data = data.convert(\'RGB\')\n        data = self.transforms(data)\n        return data.float(),sample_path\n\n    def __len__(self):\n        return len(self.image_paths)\n#infer\nimport tqdm\nimport timm\n@torch.no_grad()\ndef infer_wheather(model):\n    pres_dic = {\'id\':[],\'weather\':[]}\n    cnt = 0\n    all_output=[]\n    all_pathes=[]\n    for data in

In [17]:
'''
pres_dic = {'id':[],'weather':[]}
for i in range(len(all_output_2_1)):
    outputs=torch.pow(all_output_2[i]*all_output_2_1[i],1/2)
    _, preds = torch.max(outputs.data, 1)
    cls = preds.cpu().numpy().tolist()
    image_names=all_pathes_1[i]
    #print(image_names)
    pres_dic['id']+=image_names
    pres_dic['weather']+=cls
    #
df_sub=pd.DataFrame(pres_dic)
df_sub.to_csv(opt.submit_csv_dir+'10_18TTA.csv',index=False)
'''

In [7]:
!wget -nv -O kesci_submit https://cdn.kesci.com/submit_tool/v4/kesci_submit&&chmod +x kesci_submit

2020-10-22 13:28:24 URL:https://cdn.kesci.com/submit_tool/v4/kesci_submit [7357446/7357446] -> "kesci_submit" [1]


In [13]:
!./kesci_submit -token a030e738c18bd9f2 -file /home/kesci/work/submit/10_20v2Mean3.csv

Kesci Submit Tool 4.0.0

> 已验证Token
> 提交文件 /home/kesci/work/submit/10_20v2Mean3.csv (1204.74 KiB), Target Qiniu
> 已上传 100 %
> 文件已上传        
> 错误: 提交失败
> 服务器响应 400 {"message":"今日提交次数达到上限"}


In [37]:
#import timm
#model = timm.create_model('swsl_resnext101_32x8d', pretrained=True)
#!wget https://dl.fbaipublicfiles.com/semiweaksupervision/model_files/semi_weakly_supervised_resnext101_32x8-b4712904.pth

--2020-10-17 03:12:07--  https://dl.fbaipublicfiles.com/semiweaksupervision/model_files/semi_weakly_supervised_resnext101_32x8-b4712904.pth
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.74.142, 104.22.75.142, 172.67.9.4, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.74.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 356056638 (340M) [application/octet-stream]
Saving to: ‘semi_weakly_supervised_resnext101_32x8-b4712904.pth’

-b4712904.pth        19%[==>                 ]  64.54M  40.1KB/s    eta 50m 11s